### pip install & import

In [1]:
%pip install sklearn 
%pip install seaborn
%pip install cmake
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import pandas_profiling
import datetime

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

# 2003~2024년 데이터 전처리


### 데이터 값 읽어오기

In [34]:
# 데이터 값 읽어오기 
data_file = "/Users/yoojunghyun/Desktop/WebProj/MovieDL/data/movie_data/KOBIS_개봉일람_2023-03-23.csv"
df = pd.read_csv(data_file, encoding='UTF-8',index_col = "순번")

### 생략된 내용을 추가로 보는 코드

In [4]:
# row 생략 없이 출력
pd.set_option('display.max_rows', 200)
# col 생략 없이 출력
pd.set_option('display.max_columns', 200)

In [35]:
# 널값 확인
df.isnull()

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
순번,,,,,,,,,,,,,,,,,
1,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19900,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
19901,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
19902,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [36]:
# 빈값 확인

df.isnull().sum()

영화명            0
감독          1298
제작사        12618
수입사         8178
배급사           76
개봉일            2
영화유형           0
영화형태           1
국적             1
전국 스크린수        0
전국 매출액        44
전국 관객수         0
서울 매출액        50
서울 관객수         0
장르            71
등급             5
영화구분           0
dtype: int64

In [37]:
# 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19904 entries, 1 to 19904
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      19904 non-null  object
 1   감독       18606 non-null  object
 2   제작사      7286 non-null   object
 3   수입사      11726 non-null  object
 4   배급사      19828 non-null  object
 5   개봉일      19902 non-null  object
 6   영화유형     19904 non-null  object
 7   영화형태     19903 non-null  object
 8   국적       19903 non-null  object
 9   전국 스크린수  19904 non-null  object
 10  전국 매출액   19860 non-null  object
 11  전국 관객수   19904 non-null  object
 12  서울 매출액   19854 non-null  object
 13  서울 관객수   19904 non-null  object
 14  장르       19833 non-null  object
 15  등급       19899 non-null  object
 16  영화구분     19904 non-null  object
dtypes: object(17)
memory usage: 2.7+ MB


### 결측치 데이터 처리하기

In [38]:
# 제작사, 수입사는 결측치가 너무 많아, drop
# 나머지 결측치는 기타로 변경

df = df.drop(['제작사', '수입사'], axis=1)
df['감독'].fillna('기타', inplace=True)
df['장르'].fillna('기타', inplace=True)
df['배급사'].fillna('기타', inplace=True)
df['영화형태'].fillna('기타', inplace=True)

# 한개가 결측치가 나와서 기타로 치환
df['국적'].fillna('기타', inplace=True)

# 서울 매출액 결측치 제거(사용을 안하니깨문에 0으로 변경)
df['서울 매출액'].fillna(0, inplace=True)
# 남은 결측치 제거

for i in df['개봉일'][df['개봉일'].isnull()].index:
    df = df.drop(i)

for i in df['전국 매출액'][df['전국 매출액'].isnull()].index:
    df = df.drop(i)

for i in df['등급'][df['등급'].isnull()].index:
    df = df.drop(i)

In [39]:
# 최종확인

df.isnull().sum()

영화명        0
감독         0
배급사        0
개봉일        0
영화유형       0
영화형태       0
국적         0
전국 스크린수    0
전국 매출액     0
전국 관객수     0
서울 매출액     0
서울 관객수     0
장르         0
등급         0
영화구분       0
dtype: int64

### 범주형 데이터값 확인

In [40]:
# 범주형 데이터 값 확인

category_cols = ['영화명','감독', '배급사', '영화유형','영화형태','국적','장르','등급','영화구분',]

# 범주형 데이터의 unique 확인
for col in category_cols:
    print('컬럼 [{}] UNIQUE : {}'.format(col, len(df[col].unique())))

컬럼 [영화명] UNIQUE : 19484
컬럼 [감독] UNIQUE : 7876
컬럼 [배급사] UNIQUE : 1357
컬럼 [영화유형] UNIQUE : 1
컬럼 [영화형태] UNIQUE : 4
컬럼 [국적] UNIQUE : 72
컬럼 [장르] UNIQUE : 21
컬럼 [등급] UNIQUE : 18
컬럼 [영화구분] UNIQUE : 2


### 복수의 감독, 배급사 데이터를 한개 데이터로 변경, (연령)등급 4가지로 정리

In [41]:
# 다수의 감독 데이터 변경
for idx, row in df.iterrows():
    if ',' in row['감독']:
        directors = row['감독'].split(',')
        # 양 끝 공백 제거 후 정렬
        directors.sort(key=lambda x : x.strip())
        df['감독'][idx] = directors[0].strip()

In [42]:
# 다수의 배급사를 앞에 있는 배급사로 취급

for idx, row in df.iterrows():
    if ',' in row['배급사']:
        distributor = row['배급사'].split(',')
        # 양 끝 공백 제거 후 변경
        df['배급사'][idx] = distributor[0].strip()

In [43]:
# 영화유형은 두개 뿐이라 제거

df.drop('영화유형', axis=1, inplace=True)

In [44]:
# 등급 데이터 전처리

for idx, row in df.iterrows():
    if ',' in row['등급']:
        ratings = row['등급'].split(',')
        # 전체, 12, 15, 청소년관람불가 순
        if '전체' in ratings:
            df['등급'][idx] = '전체관람가'
        elif '12' in ratings:
            df['등급'][idx] = '12세관람가'
        elif '15' in ratings:
            df['등급'][idx] = '15세관람가'
        else:
            df['등급'][idx] = '청소년관람불가'
    elif df['등급'][idx] == '18세관람가' or df['등급'][idx] == '제한상영가': 
        df['등급'][idx] = '청소년관람불가'
    else:
        df['등급'][idx] = df['등급'][idx].replace('이상', '')

### 전국 매출액, 스크린수, 관객수 int 타입변경 

In [45]:
# 전국 매출액, 스크린 수, 관객수를 확인을 위해 int형으로 변경

df['전국 매출액'] = df['전국 매출액'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 매출액'] = df['전국 매출액'].astype('int64')

df['전국 스크린수'] = df['전국 스크린수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 스크린수'] = df['전국 스크린수'].astype('int64')

df['전국 관객수'] = df['전국 관객수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 관객수'] = df['전국 관객수'].astype('int64')


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19854 entries, 1 to 19904
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      19854 non-null  object
 1   감독       19854 non-null  object
 2   배급사      19854 non-null  object
 3   개봉일      19854 non-null  object
 4   영화형태     19854 non-null  object
 5   국적       19854 non-null  object
 6   전국 스크린수  19854 non-null  int64 
 7   전국 매출액   19854 non-null  int64 
 8   전국 관객수   19854 non-null  int64 
 9   서울 매출액   19854 non-null  object
 10  서울 관객수   19854 non-null  object
 11  장르       19854 non-null  object
 12  등급       19854 non-null  object
 13  영화구분     19854 non-null  object
dtypes: int64(3), object(11)
memory usage: 2.8+ MB


In [47]:
# 스크린 수로 하였으나, 몇몇 역대 영화들이 사라져서 전국 관객수 10000명 기준으로 변경

movie_resize = df[(df['전국 관객수'] > 10000)]
movie_resize.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4835 entries, 1 to 4867
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      4835 non-null   object
 1   감독       4835 non-null   object
 2   배급사      4835 non-null   object
 3   개봉일      4835 non-null   object
 4   영화형태     4835 non-null   object
 5   국적       4835 non-null   object
 6   전국 스크린수  4835 non-null   int64 
 7   전국 매출액   4835 non-null   int64 
 8   전국 관객수   4835 non-null   int64 
 9   서울 매출액   4835 non-null   object
 10  서울 관객수   4835 non-null   object
 11  장르       4835 non-null   object
 12  등급       4835 non-null   object
 13  영화구분     4835 non-null   object
dtypes: int64(3), object(11)
memory usage: 566.6+ KB


In [48]:
# 값 내보내기
movie_resize.to_csv('./movie_resize2.csv',index=False, encoding="utf-8-sig")

### 배급사 추가로 변경 더 해야 해yo(성진오빠)

In [49]:
# 점수 처리를 위해 변경

movie_resize['배급사'] = movie_resize['배급사'].str.replace('CJ ENM','(주)씨제이이엔엠')

/var/folders/_2/wq0k1swx2kx33qmjy7b4kw080000gn/T/ipykernel_29172/3730608000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize['배급사'] = movie_resize['배급사'].str.replace('CJ ENM','(주)씨제이이엔엠')


# 데이터 리사이징 하기
전국 스크린수, 전국 관객수 좌로 편향된 그래프 pr_report.html(태현폴더) <br>
때문에 아래의 리사이징 실행


In [50]:
# 국적 데이터 변경
for idx, row in movie_resize.iterrows():
    if  row['국적'] not in ['미국', '한국', '일본', '영국', '프랑스','중국','독일']:
        movie_resize['국적'][idx] = '기타'

/var/folders/_2/wq0k1swx2kx33qmjy7b4kw080000gn/T/ipykernel_29172/1179239548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize['국적'][idx] = '기타'


In [51]:
# 값 확인

movie_resize['국적'].value_counts()

미국     1878
한국     1543
일본      443
기타      399
영국      212
프랑스     201
중국       93
독일       66
Name: 국적, dtype: int64

In [52]:
# 스크린 수로 하였으나, 몇몇 역대 영화들이 사라져서 전국 관객수 10000명 기준으로 변경

movie_resize = df[(df['전국 관객수'] > 10000)]
movie_resize.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4835 entries, 1 to 4867
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      4835 non-null   object
 1   감독       4835 non-null   object
 2   배급사      4835 non-null   object
 3   개봉일      4835 non-null   object
 4   영화형태     4835 non-null   object
 5   국적       4835 non-null   object
 6   전국 스크린수  4835 non-null   int64 
 7   전국 매출액   4835 non-null   int64 
 8   전국 관객수   4835 non-null   int64 
 9   서울 매출액   4835 non-null   object
 10  서울 관객수   4835 non-null   object
 11  장르       4835 non-null   object
 12  등급       4835 non-null   object
 13  영화구분     4835 non-null   object
dtypes: int64(3), object(11)
memory usage: 566.6+ KB


In [53]:
# 스크린 수로 하였으나, 몇몇 역대 영화들이 사라져서 전국 관객수 10000명 기준으로 변경

movie_resize2 = df[(df['전국 관객수'] > 1000) & (df['전국 스크린수'] > 50)& (df['전국 관객수'] < 11000000)]
movie_resize2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4148 entries, 25 to 7927
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      4148 non-null   object
 1   감독       4148 non-null   object
 2   배급사      4148 non-null   object
 3   개봉일      4148 non-null   object
 4   영화형태     4148 non-null   object
 5   국적       4148 non-null   object
 6   전국 스크린수  4148 non-null   int64 
 7   전국 매출액   4148 non-null   int64 
 8   전국 관객수   4148 non-null   int64 
 9   서울 매출액   4148 non-null   object
 10  서울 관객수   4148 non-null   object
 11  장르       4148 non-null   object
 12  등급       4148 non-null   object
 13  영화구분     4148 non-null   object
dtypes: int64(3), object(11)
memory usage: 486.1+ KB


In [54]:
# 값 내보내기
movie_resize.to_csv('./movie_resize2.csv',index=False, encoding="utf-8-sig")

In [55]:
movie_resize2.drop('서울 매출액', axis=1, inplace=True)
movie_resize2.drop('서울 관객수', axis=1, inplace=True)


/var/folders/_2/wq0k1swx2kx33qmjy7b4kw080000gn/T/ipykernel_29172/3840169299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize2.drop('서울 매출액', axis=1, inplace=True)
/var/folders/_2/wq0k1swx2kx33qmjy7b4kw080000gn/T/ipykernel_29172/3840169299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize2.drop('서울 관객수', axis=1, inplace=True)


In [56]:
movie_total_people = pd.read_csv('/Users/yoojunghyun/Desktop/WebProj/MovieDL/taehyeon/movie_total_people.csv',index_col='연도',encoding='UTF-8')
movie_total_people

,개봉편수,상영편수,매출액,관객수
연도,,,,
2004,280,300,"440,728,906,206","69,254,626"
2005,306,415,"780,364,979,467","123,352,059"
2006,351,482,"892,442,138,000","144,256,035"
2007,393,611,"965,486,624,538","152,022,992"
2008,380,804,"961,453,297,251","147,428,639"
2009,362,714,"1,083,196,004,950","155,398,654"
2010,430,797,"1,157,254,738,250","147,759,214"
2011,442,943,"1,235,799,464,200","159,724,465"
2012,641,"1,231","1,455,140,354,435","194,890,587"


In [57]:
movie_total_people['관객수'] = movie_total_people['관객수'].str.replace(pat=r',', repl=r'', regex=True)
movie_total_people['관객수'] = movie_total_people['관객수'].astype('int64')

In [58]:
movie_total_people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 2004 to 2023
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   개봉편수    20 non-null     object
 1   상영편수    20 non-null     object
 2   매출액     20 non-null     object
 3   관객수     20 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 800.0+ bytes


In [59]:
movie_final['총 관객수'] = [0 for _ in range(len(movie_final))]
movie_final


NameError: name 'movie_final' is not defined

In [60]:
# 관객 비율 구하기 
movie_final['관객비율'] = (movie_final['전국 관객수'] / movie_final['총 관객수'])
# 2003 1997 총 관객수 0명 데이터 때문에 위 코드 실행
movie_final[movie_final['관객비율']==np.inf] = 0 

NameError: name 'movie_final' is not defined

# 예지언니 크롤링 데이터 머지

# 리사이징 된 데이터에서 top 200 생성

### top 200 생성
movie_resize_ranking.csv

In [61]:
# top200 만듬

col = ['순번','영화명','감독','배급사','개봉일']
movie_resize.reset_index(drop=False,inplace=True)
movie_resize[col][:200].to_csv('./movie_resize_ranking.csv', encoding="utf-8-sig")

In [62]:
# top 200 불러오기

movie_resize_ranking = pd.read_csv('./movie_resize_ranking.csv',index_col='순번',encoding='UTF-8')
movie_resize_ranking.drop('Unnamed: 0', axis=1, inplace=True)
movie_resize_ranking

,영화명,감독,배급사,개봉일
순번,,,,
1,명량,김한민,(주)씨제이이엔엠,2014-07-30
2,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23
3,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20
4,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17
5,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24
6,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21
7,아바타,제임스 카메론,주식회사 해리슨앤컴퍼니,2009-12-17
8,베테랑,류승완,(주)씨제이이엔엠,2015-08-05
9,괴물,봉준호,(주)쇼박스,2006-07-27


In [63]:
# 점수화를 위해 개봉일을 datetime 타입으로 변경

movie_resize_ranking['개봉일'] = movie_resize_ranking['개봉일'].astype('datetime64')

### 점수화행 만들기

In [64]:
# 점수화행 만들기

import datetime

movie_resize2['개봉일'] = movie_resize2['개봉일'].astype('datetime64')

movie_final = movie_resize2.copy()
movie_final['감독_흥행'] = [0 for _ in range(len(movie_resize2))]
movie_final['배급사_흥행'] = [0 for _ in range(len(movie_resize2))]
movie_final['주연배우_흥행'] = [0 for _ in range(len(movie_resize2))]

/var/folders/_2/wq0k1swx2kx33qmjy7b4kw080000gn/T/ipykernel_29172/742394046.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize2['개봉일'] = movie_resize2['개봉일'].astype('datetime64')


In [65]:
# 순번을 쓰기 위해 인덱스 풀기

movie_resize_ranking.reset_index(drop=False,inplace=True)

### 감독, 배급사 점수화 코드

In [66]:
# 감독 점수화 코드

for idx, row in movie_final.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['감독'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['감독'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final.loc[idx, '감독_흥행'] = score

In [67]:
# 배급사 정수화 코드 
for idx, row in movie_final.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['배급사'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['배급사'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final.loc[idx, '배급사_흥행'] = score

In [68]:
# 값 확인
(movie_final['배급사_흥행'] == 0).value_counts()

True     2385
False    1763
Name: 배급사_흥행, dtype: int64

### 날짜 데이터 datetime으로 변경

In [69]:
# 날짜 데이터를 datetime으로 변경

import datetime
movie_final['연도'] = movie_final['개봉일'].dt.year
movie_final['월'] = movie_final['개봉일'].dt.month

movie_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4148 entries, 25 to 7927
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   영화명      4148 non-null   object        
 1   감독       4148 non-null   object        
 2   배급사      4148 non-null   object        
 3   개봉일      4148 non-null   datetime64[ns]
 4   영화형태     4148 non-null   object        
 5   국적       4148 non-null   object        
 6   전국 스크린수  4148 non-null   int64         
 7   전국 매출액   4148 non-null   int64         
 8   전국 관객수   4148 non-null   int64         
 9   장르       4148 non-null   object        
 10  등급       4148 non-null   object        
 11  영화구분     4148 non-null   object        
 12  감독_흥행    4148 non-null   int64         
 13  배급사_흥행   4148 non-null   int64         
 14  주연배우_흥행  4148 non-null   int64         
 15  연도       4148 non-null   int64         
 16  월        4148 non-null   int64         
dtypes: datetime64[ns](1), int64(8), 

### 시즌 데이터 변경

In [70]:
# 시즌 데이터 변경

season = {12 : '겨울',
          1 : '겨울',
          2 : '겨울',
          3 : '봄',
          4 : '봄',
          5 : '봄',
          6 : '여름',
          7 : '여름',
          8 : '여름',
          9 : '가을',
          10 : '가을',
          11 : '가을'}

movie_final['계절'] = movie_final[['월']].apply(lambda x : x.map(season))

### moive final

In [71]:

movie_final

,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,장르,등급,영화구분,감독_흥행,배급사_흥행,주연배우_흥행,연도,월,계절
순번,,,,,,,,,,,,,,,,,,
25,아바타: 물의 길,제임스 카메론,월트디즈니컴퍼니코리아 유한책임회사,2022-12-14,장편,미국,2809,137325939643,10779198,액션,12세관람가,일반영화,10,10,0,2022,12,겨울
26,어벤져스: 에이지 오브 울트론,조스 웨던,월트디즈니컴퍼니코리아 유한책임회사,2015-04-23,장편,미국,1843,88582586366,10494499,액션,12세관람가,일반영화,8,8,0,2015,4,봄
27,기생충,봉준호,(주)씨제이이엔엠,2019-05-30,장편,한국,1948,87459829095,10313086,드라마,15세관람가,일반영화,10,10,0,2019,5,봄
28,겨울왕국,제니퍼 리,소니픽쳐스릴리징월트디즈니스튜디오스코리아(주),2014-01-16,장편,미국,1010,82461504400,10296101,애니메이션,전체관람가,일반영화,0,9,0,2014,1,겨울
29,인터스텔라,크리스토퍼 놀란,워너브러더스 코리아(주),2014-11-06,장편,미국,1342,82274331200,10273803,SF,12세관람가,일반영화,8,8,0,2014,11,가을
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7763,앙상블,정형석,(주)스톰픽쳐스코리아,2020-11-05,장편,한국,52,10376610,1140,드라마,12세관람가,독립/예술영화,0,0,0,2020,11,가을
7781,퍼펙트 스틸,기타,(주)팝엔터테인먼트,2021-10-28,장편,영국,66,10073100,1122,범죄,15세관람가,일반영화,0,0,0,2021,10,가을
7808,신 테니스의 왕자 효테이 vs 릿카이 : 게임 오브 퓨처 전편,카와구치 케이이치로,(주)얼리버드픽쳐스,2021-03-24,장편,일본,60,8850500,1102,애니메이션,전체관람가,일반영화,0,0,0,2021,3,봄


# 최종 데이터 불러오기
위에코드를 매번 실행하지 않고 파일로 불러와서 실행 

In [12]:
# 데이터 불러오기(파일 미완성.movie_resize2와 이름이 다름 )
df = pd.read_csv('./movie_resize2.csv',index_col=0,encoding='UTF-8')
df.reset_index(drop=False,inplace=True)
df

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...
4637,나의 어머니,난니 모레티,(주)티캐스트,2015-08-20,장편,이탈리아,30,76569151,10031,드라마,12세관람가,독립/예술영화
4638,타이페이 카페 스토리,샤오 야 췐,찬란,2011-07-07,장편,대만,10,76832900,10023,드라마,전체관람가,독립/예술영화
4639,테라 3D: 인류 최후의 전쟁,아리스토메니스 치바스,주식회사 마운틴픽쳐스,2010-11-04,장편,미국,48,106210549,10023,애니메이션,전체관람가,일반영화
4640,글로리아 벨,세바스찬 렐리오,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,2019-06-06,장편,미국,48,77962520,10021,드라마,15세관람가,독립/예술영화


# ---전처리---

## 원핫인코딩 

In [72]:
# 원핫인코딩

movie_labels = pd.get_dummies(movie_final,columns = ['국적','장르','등급','계절'])

In [73]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사','영화형태','개봉일','전국 스크린수', '전국 매출액','서울 매출액','서울 관객수','영화구분','연도','월','순번','주연배우_흥행']

movie_DL = movie_labels.drop(not_used, axis=1)

KeyError: "['서울 매출액', '서울 관객수', '순번'] not found in axis"

In [74]:
# 사용하는 수치형 데이터 히트맵

digital_cols2 = ['전국 관객수','감독_흥행', '배급사_흥행']
plt.rcParams['font.family'] ='Malgun Gothic'

plt.rcParams['axes.unicode_minus'] =False
plt.figure(figsize=(12, 7))
sns.heatmap(movie_DL[digital_cols2].corr(), annot=True, fmt='.2f')
plt.show()

NameError: name 'movie_DL' is not defined

<Figure size 1200x700 with 0 Axes>

In [75]:
# 입력 컬럼 확인(값을 하나 빼야한다 => 전국 관객수로 쓰려고)
len(movie_DL.columns)

NameError: name 'movie_DL' is not defined

In [76]:
# 독립 변수와 종속 변수 나눔

독립 = movie_DL[['감독_흥행', '배급사_흥행', '국적_기타', '국적_독일', '국적_미국',
       '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF', '장르_가족', '장르_공연',
       '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬',
       '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_성인물(에로)', '장르_스릴러',
       '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
       '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
       '계절_봄', '계절_여름']] 
종속 = movie_DL[['전국 관객수']] 

NameError: name 'movie_DL' is not defined

## 라벨링

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()

movie_labels = movie_final.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final['계절'])
movie_labels.head(3)


In [ ]:
# 독립 변수와 종속 변수 나눔

독립 = movie_DL[['국적', '장르', '등급', '감독_흥행',
               '배급사_흥행', '계절', '관객비율', '전국 스크린수', '전국 매출액']]
종속 = movie_DL[['전국 관객수']] 

## train, test 데이터 나누기

In [2]:
# train, test 데이터 나눔

from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(독립,종속,test_size=0.25,random_state=42)

ModuleNotFoundError: No module named 'sklearn'

## 스케일링... 
...어디서 할까...<br>
라벨링 때문에 바로 스케일링을 따로 해줘야함 (전국스크린수, 전국매출액)

# 딥러닝 


In [ ]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[39])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)


Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
sgd = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.5)
model.compile(optimizer=sgd,loss='MAE')

In [ ]:
# 훈련

model.fit(train_input,train_target,batch_size=128,epochs=10,validation_data=(test_input,test_target))

Epoch 1/100
38/38 [==============================] - 0s 5ms/step - loss: 619277.2500
Epoch 2/100
38/38 [==============================] - 0s 4ms/step - loss: 606993.0000
Epoch 3/100
38/38 [==============================] - 0s 5ms/step - loss: 599592.3125
Epoch 4/100
38/38 [==============================] - 0s 5ms/step - loss: 595743.0625
Epoch 5/100
38/38 [==============================] - 0s 4ms/step - loss: 595327.1250
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 594885.1250
Epoch 7/100
38/38 [==============================] - 0s 4ms/step - loss: 590020.5625
Epoch 8/100
38/38 [==============================] - 0s 3ms/step - loss: 589700.3125
Epoch 9/100
38/38 [==============================] - 0s 3ms/step - loss: 587818.7500
Epoch 10/100
38/38 [==============================] - 0s 4ms/step - loss: 587813.2500
Epoch 11/100
38/38 [==============================] - 0s 4ms/step - loss: 582806.0000
Epoch 12/100
38/38 [==============================] - 0s 4ms/st

In [ ]:
# 확인용도
model.predict(train_input[:5])

1/1 [==============================] - 0s 137ms/step


array([[28853.379],
       [69694.68 ],
       [28990.406],
       [25120.441],
       [92417.41 ]], dtype=float32)

In [ ]:
# 여기랑 비슷한지 보기
train_target[:5]

,전국 관객수
3247,44231
1161,600067
4262,15576
3071,53744
1743,274628


In [ ]:
# 값 스케일링

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 랜덤포레스트 돌려보기

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8170010264946882 0.15534713748275336


In [ ]:
# 중요 특성 뽑기
rf.fit(train_input,train_target)
print(rf.feature_importances_)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_14704\577101422.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


[1.96813970e-01 3.55908596e-01 0.00000000e+00 7.06708476e-04
 1.74856009e-04 1.74090864e-02 8.54422830e-04 1.60430721e-03
 5.24058985e-04 6.92586489e-04 3.39190989e-02 1.19412744e-02
 5.98020352e-04 8.28959287e-05 2.36682798e-03 2.33287048e-05
 8.91602331e-04 1.99946004e-02 9.04713515e-03 8.90093937e-04
 2.62617167e-03 1.10334319e-02 2.20024349e-02 2.94747967e-03
 1.62065512e-07 2.97657198e-03 8.96384072e-03 3.57557677e-02
 1.13193184e-02 1.56141205e-02 2.16046910e-02 1.63842063e-02
 3.65131739e-02 2.58119565e-02 1.20582740e-02 1.11598456e-02
 2.40559232e-02 3.38250170e-02 1.91485361e-02 3.17556059e-02]


In [ ]:
# 간단한 랜덤포레스트

rf = RandomForestRegressor(n_jobs=-1,oob_score=True)
rf.fit(train_input,train_target)
print(rf.oob_score_)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_14704\762423999.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


0.14506523696259876


In [ ]:
# 익스트림 ~~
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9188274409933219 -0.0142198884575655


In [ ]:
# 특성 확인
et.fit(train_input,train_target)
print(et.feature_importances_)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_14704\1418952811.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  et.fit(train_input,train_target)


[1.80465958e-01 3.22064469e-01 0.00000000e+00 1.71247765e-03
 1.16945937e-04 2.23964121e-02 6.90403527e-04 6.76768643e-04
 5.49644738e-04 5.08610282e-04 4.13213161e-02 1.23125242e-02
 4.26241579e-04 1.87999346e-04 1.96620388e-03 4.37239683e-05
 1.17439397e-03 1.94371957e-02 8.80247740e-03 8.66204932e-04
 3.13151095e-03 9.43041454e-03 2.49148513e-02 3.72997720e-03
 1.39186747e-08 2.22812567e-03 5.10388005e-03 4.33887641e-02
 1.54831642e-02 1.71412258e-02 2.28737404e-02 1.82371194e-02
 4.18198610e-02 2.97802910e-02 1.74316031e-02 1.45242826e-02
 2.78636621e-02 3.39551485e-02 2.21870936e-02 3.10552997e-02]


In [ ]:
# 그레이디션~~
# 매출액 관객비율 값을 넣기전에 나왔던 예측값

from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.5222334360571009 0.2429401676071068


In [ ]:
pip install cmake

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 33.0/33.0 MB 29.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# xgboost 임

from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

xgb = XGBRFRegressor(tree_method='hist',random_state=42)
scores = cross_validate(xgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.5113390209278604 0.2258013878224463
